# 🛡️ Security Audit Dashboard
This notebook evaluates a model across three dimensions: **Robustness**, **Privacy**, and **Extraction Resistance**.

In [ ]:
import numpy as np

def calculate_metrics(clean_acc, robust_acc, epsilon, queries):
    # 1. Robustness Calculation
    rob_gap = clean_acc - robust_acc
    
    # 2. Privacy Interpretation
    privacy_status = "Strong" if epsilon <= 1.0 else "Weak"
    
    # 3. Extraction Cost (Assuming $0.01 per API call)
    ext_cost = queries * 0.01
    
    print(f"--- Security Report ---")
    print(f"Robustness Gap: {rob_gap:.2%} (Lower is better)")
    print(f"Privacy ($\\epsilon$): {epsilon} ({privacy_status})")
    print(f"Extraction Cost: ${ext_cost:,.2f}")
    
# Example Simulation: Model B from the demo
calculate_metrics(clean_acc=0.92, robust_acc=0.85, epsilon=1.0, queries=10000)

## ⚖️ Acceptance Criteria Framework
Use this checklist to validate deployment:
- **Robust Accuracy** > 85%?
- **Epsilon ($\\epsilon$)** <= 1.0?
- **Extraction Cost** > $5,000?